# Training and Evaluation of GNNs and LLMs
In this notebook, we train the models on the [MovieLens Dataset](https://movielens.org/) after the Pytorch Geometrics Tutorial on [Link Prediction](https://colab.research.google.com/drive/1xpzn1Nvai1ygd_P5Yambc_oe4VBPK_ZT?usp=sharing#scrollTo=vit8xKCiXAue).

First we import all of our dependencies.

The **GraphRepresentationGenerator** manages and trains a GNN model. Its most important interfaces include
**the constructor**, which defines the GNN architecture and loads the pre-trained GNN model if it is already on the hard disk,
**the training method**, which initializes the training on the GNN model and
**the get_embedding methods**, which represent the inference interface to the GNN model and return the corresponding embeddings in the dimension defined in the constructor for given user movie node pairs.

**The MovieLensLoader** loads and manages the data sets. The most important tasks include **saving and (re)loading and transforming** the data sets.

**PromptEncoderOnlyClassifier** and **VanillaEncoderOnlyClassifier** each manage a **prompt (model) LLM** and a **vanilla (model) LLM**. An EncoderOnlyClassifier (ClassifierBase) provides interfaces for training and testing an LLM model.
PromptEncoder and VanillaEncoder differ from their DataCollectors. DataCollectors change the behavior of the models during training and testing and allow data points to be created at runtime. With the help of these collators, we **create non-existent edges on the fly**.

In [1]:
from graph_representation_generator import GraphRepresentationGenerator
from dataset_manager import (
    MovieLensManager,
    INPUT_EMBEDS_REPLACE_KGE_DIMENSION,
    ROOT,
)
from llm_manager import InputEmbedsReplaceClassifier

In [2]:
EPOCHS = 4
BATCH_SIZE_KGE = 128000
BATCH_SIZE_LLM = 256

We define in advance which **Knowledge Graph Embedding Dimension (KGE_DIMENSION)** the GNN encoder has. We want to determine from which output dimension the GNN encoder can produce embeddings that lead to a significant increase in performance *without exceeding the context length of the LLMs*. In the original tutorial, the KGE_DIMENSION was $64$.

In [3]:
kg_manager = MovieLensManager()

llm_df = kg_manager.llm_df.merge(kg_manager.target_df[["id", "prompt_feature_title", "prompt_feature_genres"]].rename(columns={"id": "target_id"}), on = "target_id")
llm_df

First we load the MovieLensLoader, which downloads the Movie Lens dataset (https://files.grouplens.org/datasets/movielens/ml-32m.zip) and prepares it to be used on GNN and LLM. We also pass the embedding dimensions that we will assume we are training with. First time takes approx. 30 sec.

In [4]:
kg_manager.data

HeteroData(
  source={ node_id=[200948] },
  target={
    node_id=[87585],
    x=[87585, 20],
  },
  (source, edge, target)={ edge_index=[2, 32000204] },
  (target, rev_edge, source)={ edge_index=[2, 32000204] }
)

Next, we initialize the GNN trainers (possible on Cuda), one for each KGE_DIMENSION.
A GNN trainer manages a model and each model consists of an **encoder and classifier** part.

**The encoder** is a parameterized *Grap Convolutional Network (GCN)* with a *2-layer GNN computation graph* and a single *ReLU* activation function in between.

**The classifier** applies the dot-product between source and destination kges to derive edge-level predictions.

In [5]:
graph_representation_generator_input_embeds_replace = GraphRepresentationGenerator(
    kg_manager.data,
    kg_manager.gnn_train_data,
    kg_manager.gnn_val_data,
    kg_manager.gnn_test_data,
    hidden_channels=INPUT_EMBEDS_REPLACE_KGE_DIMENSION,
    kge_dimension=INPUT_EMBEDS_REPLACE_KGE_DIMENSION,
    force_recompute=False,
)

loading pretrained model
Device: 'cpu'


Next we produce the KGEs for every edge in the dataset. These embeddings can then be used for the LLM on the link-prediction task.

Next we initialize the vanilla encoder only classifier. This classifier does only use the NLP part of the prompt (no KGE) for predicting if the given link exists.

In [6]:
INPUT_EMBEDS_REPLACE_ROOT = f"{ROOT}/llm/input_embeds_replace"

In [7]:
input_embeds_replace_bert_classifier = InputEmbedsReplaceClassifier(
    kg_manager,
    graph_representation_generator_input_embeds_replace.get_embeddings,
    root_path=INPUT_EMBEDS_REPLACE_ROOT,
)

Some weights of InputEmbedsReplaceBertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


device cuda


In [8]:
dataset_embedding = kg_manager.generate_input_embeds_replace_embedding_dataset(
    input_embeds_replace_bert_classifier.tokenizer.sep_token,
    input_embeds_replace_bert_classifier.tokenizer.pad_token,
    input_embeds_replace_bert_classifier.tokenize_function,
)

Loading dataset from disk:   0%|          | 0/28 [00:00<?, ?it/s]

In [9]:
input_embeds_replace_bert_classifier.train_model_on_data(
    dataset_embedding, epochs=EPOCHS, batch_size=BATCH_SIZE_LLM
)
graph_representation_generator_input_embeds_replace.save_model()

  0%|          | 0/120004 [00:00<?, ?it/s]

c:\Users\MARS\.conda\envs\hauptprojekt\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 0.7654, 'grad_norm': 3.0944459438323975, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}
{'loss': 0.761, 'grad_norm': 2.7295455932617188, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.0}
{'loss': 0.7577, 'grad_norm': 2.8563897609710693, 'learning_rate': 3e-06, 'epoch': 0.0}
{'loss': 0.7525, 'grad_norm': 2.8859329223632812, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.0}
{'loss': 0.7431, 'grad_norm': 2.399235486984253, 'learning_rate': 5e-06, 'epoch': 0.0}
{'loss': 0.7275, 'grad_norm': 2.249018430709839, 'learning_rate': 6e-06, 'epoch': 0.0}
{'loss': 0.7091, 'grad_norm': 1.9441760778427124, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.0}
{'loss': 0.6957, 'grad_norm': 1.835326910018921, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.0}
{'loss': 0.6761, 'grad_norm': 1.2646160125732422, 'learning_rate': 9e-06, 'epoch': 0.0}
{'loss': 0.6588, 'grad_norm': 1.060930609703064, 'learning_rate': 1e-05, 'epoch': 0.0}
{'loss': 0.6506, 'grad_norm': 0.71541988849

  0%|          | 0/12501 [00:00<?, ?it/s]

{'eval_loss': 0.07288696616888046, 'eval_accuracy': 0.9782338860382123, 'eval_runtime': 21481.1592, 'eval_samples_per_second': 148.969, 'eval_steps_per_second': 0.582, 'epoch': 1.0}
{'loss': 0.1225, 'grad_norm': 0.4672427773475647, 'learning_rate': 3.765313294952471e-05, 'epoch': 1.0}
{'loss': 0.1398, 'grad_norm': 0.5391292572021484, 'learning_rate': 3.764894898915518e-05, 'epoch': 1.0}
{'loss': 0.1214, 'grad_norm': 0.40270090103149414, 'learning_rate': 3.764476502878565e-05, 'epoch': 1.0}
{'loss': 0.1115, 'grad_norm': 0.38638415932655334, 'learning_rate': 3.764058106841612e-05, 'epoch': 1.0}
{'loss': 0.115, 'grad_norm': 0.37232309579849243, 'learning_rate': 3.76363971080466e-05, 'epoch': 1.0}
{'loss': 0.122, 'grad_norm': 0.6339056491851807, 'learning_rate': 3.763221314767706e-05, 'epoch': 1.0}
{'loss': 0.128, 'grad_norm': 0.8826736807823181, 'learning_rate': 3.762802918730754e-05, 'epoch': 1.0}
{'loss': 0.1182, 'grad_norm': 0.3373228907585144, 'learning_rate': 3.7623845226938016e-05, 

...

{'eval_loss': 0.06834343820810318, 'eval_accuracy': 0.9789248192198798, 'eval_runtime': 21488.4532, 'eval_samples_per_second': 148.918, 'eval_steps_per_second': 0.582, 'epoch': 2.0}
{'loss': 0.1012, 'grad_norm': 0.7433093190193176, 'learning_rate': 2.5101251840942563e-05, 'epoch': 2.0}
{'loss': 0.1213, 'grad_norm': 0.542159378528595, 'learning_rate': 2.509706788057304e-05, 'epoch': 2.0}
{'loss': 0.1054, 'grad_norm': 0.3851769268512726, 'learning_rate': 2.5092883920203507e-05, 'epoch': 2.0}
{'loss': 0.1047, 'grad_norm': 0.46682208776474, 'learning_rate': 2.5088699959833985e-05, 'epoch': 2.0}
{'loss': 0.112, 'grad_norm': 0.5777125358581543, 'learning_rate': 2.5084515999464452e-05, 'epoch': 2.0}
{'loss': 0.1265, 'grad_norm': 0.5039678812026978, 'learning_rate': 2.508033203909493e-05, 'epoch': 2.0}
{'loss': 0.1021, 'grad_norm': 0.4636610150337219, 'learning_rate': 2.5076148078725397e-05, 'epoch': 2.0}
{'loss': 0.1105, 'grad_norm': 0.3751225471496582, 'learning_rate': 2.5071964118355874e-05

  0%|          | 0/12501 [00:00<?, ?it/s]

{'eval_loss': 0.0678991973400116, 'eval_accuracy': 0.9789110693058168, 'eval_runtime': 21455.624, 'eval_samples_per_second': 149.146, 'eval_steps_per_second': 0.583, 'epoch': 3.0}
{'loss': 0.1357, 'grad_norm': 0.3532634377479553, 'learning_rate': 1.2549370732360424e-05, 'epoch': 3.0}
{'loss': 0.1123, 'grad_norm': 0.5677230954170227, 'learning_rate': 1.2545186771990897e-05, 'epoch': 3.0}
{'loss': 0.098, 'grad_norm': 0.5016753673553467, 'learning_rate': 1.2541002811621369e-05, 'epoch': 3.0}
{'loss': 0.1197, 'grad_norm': 0.46990132331848145, 'learning_rate': 1.2536818851251841e-05, 'epoch': 3.0}
{'loss': 0.1135, 'grad_norm': 0.3969670236110687, 'learning_rate': 1.2532634890882314e-05, 'epoch': 3.0}
{'loss': 0.1246, 'grad_norm': 0.6401800513267517, 'learning_rate': 1.2528450930512786e-05, 'epoch': 3.0}
{'loss': 0.1193, 'grad_norm': 0.44087526202201843, 'learning_rate': 1.2524266970143259e-05, 'epoch': 3.0}
{'loss': 0.1011, 'grad_norm': 0.6345086097717285, 'learning_rate': 1.252008300977373

  0%|          | 0/12501 [00:00<?, ?it/s]

{'eval_loss': 0.06566094607114792, 'eval_accuracy': 0.9803254354660283, 'eval_runtime': 21445.5764, 'eval_samples_per_second': 149.216, 'eval_steps_per_second': 0.583, 'epoch': 4.0}
{'train_runtime': 288547.4282, 'train_samples_per_second': 106.465, 'train_steps_per_second': 0.416, 'train_loss': 0.12021305165671574, 'epoch': 4.0}
